## Comando para baixar o PDF

In [29]:
#!TOKENIZERS_PARALLELISM=true wget -O biblioteca.pdf "https://anexosportal.datalegis.net/arquivos/1878261.pdf"


--2025-04-12 18:49:53--  http://www.in.gov.br/web/dou/-/portaria-n-19-de-21-de-marco-de-2025-619527337
Resolving www.in.gov.br (www.in.gov.br)... 170.246.254.40, 200.198.216.40, 177.15.137.168, ...
Connecting to www.in.gov.br (www.in.gov.br)|170.246.254.40|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://www.in.gov.br/web/dou/-/portaria-n-19-de-21-de-marco-de-2025-619527337 [following]
--2025-04-12 18:49:54--  https://www.in.gov.br/web/dou/-/portaria-n-19-de-21-de-marco-de-2025-619527337
Connecting to www.in.gov.br (www.in.gov.br)|170.246.254.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘portaria19.html’

portaria19.html         [   <=>              ] 146.16K   175KB/s    in 0.8s    

2025-04-12 18:49:56 (175 KB/s) - ‘portaria19.html’ saved [149670]



In [ ]:
!pip install langchain faiss-cpu sentence-transformers
!pip install langchain-community
!pip install langchain_huggingface
#!pip install pypdf
!pip install chromadb
!pip install langchain_openai



In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/Colab Notebooks/IA_GEN')

In [ ]:
!wget -O portaria19.html http://www.in.gov.br/web/dou/-/portaria-n-19-de-21-de-marco-de-2025-619527337

In [30]:
from langchain_community.document_loaders import BSHTMLLoader

loader = BSHTMLLoader("portaria19.html")
dados = loader.load()
#print(data[0].page_content)

## Código para popular um Banco de Dados Vetorial

In [53]:
#from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

#dados = PyPDFLoader("biblioteca.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
textos = text_splitter.split_documents(dados)
embedding_engine = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_db = Chroma.from_documents(textos, embedding_engine, persist_directory="/content/drive/MyDrive/Colab Notebooks/IA_GEN")

## Código para montar um RAG

In [54]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
#from dotenv import load_dotenv
import os

from google.colab import userdata
#userdata.get('OPEN_API_KEY')
#load_dotenv()

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
n_documentos = 5

def format_docs(documentos):
    return "\n\n".join(documento.page_content for documento in documentos)

llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model = "gpt-4o-mini")
prompt = hub.pull("rlm/rag-prompt")

rag = (
    {
        "question": RunnablePassthrough(),
        "context": vector_db.as_retriever(k = n_documentos)
                    | format_docs
    }
    | prompt
    | llm
    | StrOutputParser()
)


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Código para perguntar ao RAG

In [55]:
prompt = "A inscrição no CAF é paga?"
rag.invoke(prompt)

'Não, a inscrição no CAF não é paga; ela é realizada mediante a apresentação espontânea das informações necessárias pelo requerente.'